In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F



In [3]:
git clone https://github.com/RCH98/FL.git

SyntaxError: invalid syntax (2075596213.py, line 1)

In [4]:
!pip3 install colabcode

8862.28s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [9]:
from colabcode import ColabCode

In [10]:
! pip3 show colabcode

Name: colabcode
Version: 0.3.0
Summary: ColabCode - Run codeserver on Colab!
Home-page: https://github.com/abhishekkrthakur/colabcode
Author: Abhishek Thakur
Author-email: abhishek4@gmail.com
License: MIT License
Location: /Users/abdirashidchorshanbiyev/Library/Python/3.9/lib/python/site-packages
Requires: jupyterlab, nest-asyncio, pyngrok, uvicorn
Required-by: 


In [12]:
ColabCode(port=10000)

FileNotFoundError: [Errno 2] No such file or directory: 'wget'

In [5]:
# Define specified CNN  architecture for CIFAR-100
class CNNCifar(nn.Module):
  def __init__(self, num_classes = 100):
    super(CNNCifar, self).__init__()
    self.conv1 = nn.Conv2d(3,64,kernel_size=5, padding=2)
    self.pool1 = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(64,64, kernel_size= 5, padding=2)
    self.pool2 = nn.MaxPool2d(2,2)
    self.fc1 = nn.Linear(64 * 8 * 8, 384)
    self.fc2 = nn.Linear(384, 192)
    self.classifier = nn.Linear(192, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.pool1(x)
    x = self.conv2(x)
    x = self.pool2(x)
    x = x.view(-1, 64 * 8 * 8)
    x = self.fc1(x)
    x = self.fc2(x)
    x = self.classifier(x)
    return x # the predicted class scores for the input image batch



NameError: name 'nn' is not defined

In [3]:
from torchvision import transforms

transform_train= transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])


In [4]:
trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)


100%|██████████| 169M/169M [00:05<00:00, 30.1MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
# how to split test dataset?
# Split training data for validation
# Split training set into train and validation
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])


In [6]:
# Create data loaders
batch_size = 32

trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)


In [7]:
# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNCifar().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0004)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)


In [8]:
# Training function
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total


In [9]:
# Evaluation function
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total


In [10]:
# Training loop
epochs = 150
train_losses, train_accs = [], []
val_losses, val_accs = [], []
test_losses, test_accs = [], []

In [ ]:
for epoch in range(epochs):
    train_loss, train_acc = train(model, trainloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, valloader, criterion, device)
    test_loss, test_acc = evaluate(model, testloader, criterion, device)

    scheduler.step()

    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print()

# Plot results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.plot(test_losses, label='Test')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Validation')
plt.plot(test_accs, label='Test')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()

Epoch 1/150
Train Loss: 4.1178, Train Acc: 8.14%
Val Loss: 3.8531, Val Acc: 11.39%
Test Loss: 3.8015, Test Acc: 13.35%

Epoch 2/150
Train Loss: 3.7305, Train Acc: 14.21%
Val Loss: 3.6253, Val Acc: 16.08%
Test Loss: 3.5410, Test Acc: 17.42%

Epoch 3/150
Train Loss: 3.5247, Train Acc: 17.44%
Val Loss: 3.4105, Val Acc: 19.10%
Test Loss: 3.3136, Test Acc: 21.55%

Epoch 4/150
Train Loss: 3.3629, Train Acc: 20.25%
Val Loss: 3.3041, Val Acc: 21.32%
Test Loss: 3.1500, Test Acc: 25.46%

Epoch 5/150
Train Loss: 3.2454, Train Acc: 22.66%
Val Loss: 3.2512, Val Acc: 22.80%
Test Loss: 3.1149, Test Acc: 25.48%

Epoch 6/150
Train Loss: 3.1208, Train Acc: 24.76%
Val Loss: 3.0779, Val Acc: 25.57%
Test Loss: 2.8817, Test Acc: 29.65%

Epoch 7/150
Train Loss: 3.0214, Train Acc: 26.92%
Val Loss: 3.0438, Val Acc: 26.84%
Test Loss: 2.8221, Test Acc: 31.07%

Epoch 8/150
Train Loss: 2.9549, Train Acc: 28.36%
Val Loss: 2.9701, Val Acc: 28.19%
Test Loss: 2.7609, Test Acc: 32.19%

Epoch 9/150
Train Loss: 2.9011, T

In [ ]:
! pip install wandb
import wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 104.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [ ]:
def main():
    # Initialize wandb
    wandb.init(project="cifar100-centralized-baseline")

    # Access hyperparameters
    config = wandb.config

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load data
    trainloader, valloader, testloader = load_data(config.batch_size)

    # Initialize model, loss function, and optimizer
    model = CNNCifar().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=config.momentum, weight_decay=config.weight_decay)

    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=config.epochs)

    # Training loop
    for epoch in range(config.epochs):
        train_loss, train_acc = train(model, trainloader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, valloader, criterion, device)
        test_loss, test_acc = evaluate(model, testloader, criterion, device)

        scheduler.step()

        # Log metrics to wandb
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "test_loss": test_loss,
            "test_acc": test_acc,
            "learning_rate": scheduler.get_last_lr()[0]
        })

        print(f"Epoch {epoch+1}/{config.epochs}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
        print()

    # Close wandb run
    wandb.finish()

# Define the sweep configuration
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    'parameters': {
        'batch_size': {'values': [64, 128, 256]},
        'learning_rate': {'min': 0.001, 'max': 0.1},
        'momentum': {'min': 0.8, 'max': 0.99},
        'weight_decay': {'min': 1e-5, 'max': 1e-3, 'distribution': 'log_uniform'},
        'epochs': {'value': 100}
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep_config, project="cifar100-centralized-baseline")

# Start the sweep
wandb.agent(sweep_id, function=main, count=10)



wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. weight_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: tejhe79i
Sweep URL: https://wandb.ai/deep_leaerning_pytorch/cifar100-centralized-baseline/sweeps/tejhe79i


wandb: Agent Starting Run: cxa0q3pu with config:
wandb: 	batch_size: 64
wandb: 	epochs: 100
wandb: 	learning_rate: 0.006548317096494973
wandb: 	momentum: 0.8269530640187834
wandb: 	weight_decay: 1.0000661323412507
wandb: Currently logged in as: s314800 (deep_leaerning_pytorch). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


...

Run 0ddugqoi errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-13-30773cf838b8>", line 12, in main
    trainloader, valloader, testloader = load_data(config.batch_size)
NameError: name 'load_data' is not defined

wandb: ERROR Run 0ddugqoi errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-13-30773cf838b8>", line 12, in main
wandb: ERROR     trainloader, valloader, testloader = load_data(config.batch_size)
wandb: ERROR NameError: name 'load_data' is not defined
wandb: ERROR 
wandb: Agent Starting Run: 5z1philw with config:
wandb: 	batch_size: 128
wandb: 	epochs: 100
wandb: 	learning_rate: 0.07343463755051535
wandb: 	momentum: 0.9277584857725204
wandb: 	weight_decay: 1.00049

Run 5z1philw errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-13-30773cf838b8>", line 12, in main
    trainloader, valloader, testloader = load_data(config.batch_size)
NameError: name 'load_data' is not defined

wandb: ERROR Run 5z1philw errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-13-30773cf838b8>", line 12, in main
wandb: ERROR     trainloader, valloader, testloader = load_data(config.batch_size)
wandb: ERROR NameError: name 'load_data' is not defined
wandb: ERROR 
wandb: Agent Starting Run: 1j7ko1to with config:
wandb: 	batch_size: 64
wandb: 	epochs: 100
wandb: 	learning_rate: 0.04216769767670046
wandb: 	momentum: 0.8895150468801913
wandb: 	weight_decay: 1.000452

Run 1j7ko1to errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-13-30773cf838b8>", line 12, in main
    trainloader, valloader, testloader = load_data(config.batch_size)
NameError: name 'load_data' is not defined

wandb: ERROR Run 1j7ko1to errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-13-30773cf838b8>", line 12, in main
wandb: ERROR     trainloader, valloader, testloader = load_data(config.batch_size)
wandb: ERROR NameError: name 'load_data' is not defined
wandb: ERROR 
wandb: Agent Starting Run: 3r9d5l6k with config:
wandb: 	batch_size: 256
wandb: 	epochs: 100
wandb: 	learning_rate: 0.0938567586017676
wandb: 	momentum: 0.8044901925730037
wandb: 	weight_decay: 1.000140

Run 3r9d5l6k errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-13-30773cf838b8>", line 12, in main
    trainloader, valloader, testloader = load_data(config.batch_size)
NameError: name 'load_data' is not defined

wandb: ERROR Run 3r9d5l6k errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-13-30773cf838b8>", line 12, in main
wandb: ERROR     trainloader, valloader, testloader = load_data(config.batch_size)
wandb: ERROR NameError: name 'load_data' is not defined
wandb: ERROR 
wandb: Agent Starting Run: p6oonf37 with config:
wandb: 	batch_size: 128
wandb: 	epochs: 100
wandb: 	learning_rate: 0.059899941691452706
wandb: 	momentum: 0.8105125950387101
wandb: 	weight_decay: 1.0004

Run p6oonf37 errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "<ipython-input-13-30773cf838b8>", line 12, in main
    trainloader, valloader, testloader = load_data(config.batch_size)
NameError: name 'load_data' is not defined

wandb: ERROR Run p6oonf37 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.10/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-13-30773cf838b8>", line 12, in main
wandb: ERROR     trainloader, valloader, testloader = load_data(config.batch_size)
wandb: ERROR NameError: name 'load_data' is not defined
wandb: ERROR 
Detected 5 failed runs in a row at start, killing sweep.
wandb: ERROR Detected 5 failed runs in a row at start, killing sweep.
wandb: To change this value set WANDB_AGENT_MAX_INITIAL_FAILURES=val


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import wandb

# Initialize wandb
wandb.init(project="centralized-baseline-cifar100", config={
    "epochs": 150,
    "batch_size": 32,
    "lr": 0.001,
    "momentum": 0.9,
    "weight_decay": 0.0004,
    "optimizer": "SGD",
    "scheduler": "CosineAnnealingLR",
})

# Get config values
config = wandb.config

# Define the CNN architecture
class CNNCifar(nn.Module):
    def __init__(self, num_classes=100):
        super(CNNCifar, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 384)
        self.fc2 = nn.Linear(384, 192)
        self.classifier = nn.Linear(192, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = x.view(-1, 64 * 8 * 8)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.classifier(x)
        return x

# Define transformations
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4865, 0.4409), (0.2673, 0.2564, 0.2761))
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

# Split training set into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])

# Create data loaders
trainloader = DataLoader(trainset, batch_size=config.batch_size, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=config.batch_size, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=config.batch_size, shuffle=False, num_workers=2)

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNCifar().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=config.lr, momentum=config.momentum, weight_decay=config.weight_decay)

# Learning rate scheduler
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=150)

# Training function
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total

# Evaluation function
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), 100. * correct / total

# Training loop
train_losses, train_accs = [], []
val_losses, val_accs = [], []
test_losses, test_accs = [], []


for epoch in range(config.epochs):
    train_loss, train_acc = train(model, trainloader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, valloader, criterion, device)
    test_loss, test_acc = evaluate(model, testloader, criterion, device)

    scheduler.step()

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_acc": train_acc,
        "val_loss": val_loss,
        "val_acc": val_acc,
        "test_loss": test_loss,
        "test_acc": test_acc,
        "learning_rate": scheduler.get_last_lr()[0]
    })

    print(f"Epoch {epoch + 1}/{config.epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%")
    print()

# Finalize wandb run
wandb.finish()

# Plot results
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train')
plt.plot(val_losses, label='Validation')
plt.plot(test_losses, label='Test')
plt.title('Loss vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accs, label='Train')
plt.plot(val_accs, label='Validation')
plt.plot(test_accs, label='Test')
plt.title('Accuracy vs. Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.tight_layout()
plt.show()


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
learning_rate,███████▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▂▂▁▁
test_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,█▄▅▃▃▂▁▃▄▂▄▂▃▄▄▁▃▂▃▃▃▃▃▂▄▃▃▁
train_loss,▁███████████████████████████
val_acc,██▂▅▂▃▄▃▁▃▇▇▂▂▂▂▃▁▃▃▃▂▁▁▂▂▁▂
val_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,28
learning_rate,0.00095
test_acc,1


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/100
Train Loss: 4.6048, Train Acc: 1.06%
Val Loss: 4.6052, Val Acc: 0.93%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 2/100
Train Loss: 4.6052, Train Acc: 1.01%
Val Loss: 4.6052, Val Acc: 1.03%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 3/100
Train Loss: 4.6052, Train Acc: 0.94%
Val Loss: 4.6052, Val Acc: 0.83%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 4/100
Train Loss: 4.6052, Train Acc: 0.85%
Val Loss: 4.6052, Val Acc: 0.86%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 5/100
Train Loss: 4.6052, Train Acc: 0.89%
Val Loss: 4.6052, Val Acc: 0.86%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 6/100
Train Loss: 4.6052, Train Acc: 0.97%
Val Loss: 4.6052, Val Acc: 0.84%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 7/100
Train Loss: 4.6052, Train Acc: 0.99%
Val Loss: 4.6052, Val Acc: 0.81%
Test Loss: 4.6052, Test Acc: 1.00%

Epoch 8/100
Train Loss: 4.6052, Train Acc: 0.93%
Val Loss: 4.6052, Val Acc: 0.81%
Test Loss: 4.605